In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pylab as plt
from matplotlib.patches import Rectangle


In [3]:
from promised_land import PromisedLandTrader

# Compare different trading strategies

## Strategy 1 - Koudie

### Inputs

In [4]:
#Read parameters
dict_read_data = {
    "interval":"1h", # Candles of 1 day
    "start_date": "2024-01-01"
                 }

In [5]:
# Signal detection
dict_detect_trades = {
    "rolling_window":6, #Window width in candles of value weighted average price (VWAP)
    "threshold":0.03, #Fraction of deviation from reference metric. (VWAP)
                     }

dict_detect_range = {"rolling_window":np.arange(1,12,1),
                     "threshold":np.linspace(0.01,0.1,10),
                    }

In [6]:
#Trading amounts
dict_trade_logic = {"initial_investment":100,
                    "trade_fraction_cash":0.2, #Fraction of cash value traded
                    "trade_fraction_volume":0.2, #Fraction of unit volume traded
                   }

## Initialisation

In [7]:
koudie = PromisedLandTrader(dict_read_data, 
                            dict_detect_trades,
                            dict_trade_logic,
                            dict_detect_range,
                            )

### Performs trades and evaluate strategy

In [ ]:
koudie.optimise_strategy()

In [ ]:
df_sp = koudie.df_strategic_points

In [ ]:
df_time_s = koudie.df_timeseries

### Optimise detections

In [ ]:
params = list(koudie.detect_config_range.keys())

In [ ]:
df_sp.sort_values("normalised_wallet_value",ascending=False).head(10)[params+["normalised_wallet_value"]]

In [ ]:
df_stat = df_sp.groupby(params)["normalised_wallet_value"].mean().to_frame().unstack()

In [ ]:
fig, ax = plt.subplots()

# Plot heatmap
img = ax.imshow(df_stat)

# Find position of maximum value
max_pos = np.unravel_index(np.nanargmax(df_stat.values), df_stat.shape)
y_max, x_max = max_pos  # row, column

# Draw red rectangle around the max cell
rect = Rectangle(
    (x_max - 0.5, y_max - 0.5),
    1, 1,
    linewidth=2,
    edgecolor="red",
    facecolor="none"
)
ax.add_patch(rect)

# Labels
ax.set_xlabel(df_stat.columns.names[1])
ax.set_ylabel(df_stat.index.name)

# Colorbar
fig.colorbar(img, ax=ax)

plt.show()


In [ ]:
normalised_wallet = df_time_s.columns[df_time_s.columns.str.contains("normalised_wallet")]

In [ ]:
df_time_s.plot(x="timestamp", y=normalised_wallet)

In [ ]:
total_wallet = df_time_s.columns[df_time_s.columns.str.contains("total_wallet_value")]

In [ ]:
df_time_s.plot(x="timestamp", y=total_wallet,label=None)

### Show strategy performance

In [ ]:
koudie.perform_analysis()